# Real time Experimentation

In [42]:
import cv2
import mediapipe as mp
import time
import numpy as np

In [43]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=True, max_num_hands=1)
mpDraw = mp.solutions.drawing_utils
color = (255, 0, 0)
thickness = 2

def crop_img(img):
    cropped_image = img
    h, w, c = img.shape
    if h < w:
        wcenter = w/2
        wmin = int(wcenter-h/2)
        wmax = int(wcenter+h/2)
        cropped_image = img[0:h, wmin:wmax]
    else:
        hcenter = h/2
        hmin = int(hcenter-w/2)
        hmax = int(hcenter+w/2)
        cropped_image = img[hmin:hmax, 0:w]

    imgRGB = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks:
        for handlms in results.multi_hand_landmarks:
            # z = max([lm.z for lm in handlms.landmark])
            h, w, c = cropped_image.shape

            xmax = max([int(lm.x*w) for lm in handlms.landmark])
            xmax = w if xmax > w else xmax

            xmin = min([int(lm.x*w) for lm in handlms.landmark])
            xmin = 0 if xmin < 0 else xmin

            ymax = max([int(lm.y*h) for lm in handlms.landmark])
            ymax = h if ymax > h else ymax

            ymin = min([int(lm.y*h) for lm in handlms.landmark])
            ymin = 0 if ymin < 0 else ymin

            xsize = int(xmax - xmin)
            ysize = int(ymax - ymin)
            
            cv2.circle(cropped_image, (int((xmin+xmax)/2), int((ymin+ymax)/2)), max(int((xsize/2)+60), int((ysize/2)+60)), color, thickness)
            
            r = max(int((xsize/2)+60), int((ysize/2)+60))
            xmin = int((xmin+xmax)/2 - r)
            xmax = int((xmin+xmax)/2 + r)

            if xmax > w:
                xmax = w
                xmin = int((xmin+xmax)/2 - r)

            if xmin < 0:
                xmin = 0
                xmax = int((xmin+xmax)/2 + r)

            ymin = int((ymin+ymax)/2 - r)
            ymax = int((ymin+ymax)/2 + r)

            if ymax > h:
                ymax = h
                ymin = int((ymin+ymax)/2 - r)

            if ymin < 0:
                ymin = 0
                ymax = int((ymin+ymax)/2 + r)

            
            cropped_image = cropped_image[ymin:ymax, xmin:xmax]

            # for id, lm in enumerate(handlms.landmark):
                #     cx, cy = int(lm.x*w), int(lm.y*h)
                #     cropped_image = cv2.circle(cropped_image, (cx, cy), 25, color, thickness)
                    
                # mpDraw.draw_landmarks(cropped_image, handlms, mpHands.HAND_CONNECTIONS)
    return cropped_image

In [44]:
cap = cv2.VideoCapture(0)
pTime = time.time()

while True:
    success, img = cap.read()

    if (success):

        cropped_image = crop_img(img)
        
        cTime = time.time()
        fps = 1/(cTime - pTime)
        pTime = cTime

        cv2.putText(cropped_image, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 255), 3) 
        cv2.imshow("image", cropped_image)
        cv2.waitKey(1)

KeyboardInterrupt: 

: 